In [21]:
import pandas as pd
import sklearn as sk 
import numpy as np
# Выборки уже поделены на отдельные таблицы, поэтому метод сплит не понадобится 

X_train = pd.read_csv('dsc_salary_2025_train.csv')
X_test = pd.read_csv('dsc_salary_2025_test.csv')

# Сразу удаляю ненужные и шумовые колонки признаков для обоих выборок
to_drop = ['Unnamed: 0', 'id', 'first_name','noise_feature_5', 'noise_feature_1', 'noise_feature_2', 'noise_feature_3', 'noise_feature_4']

X_train.drop(to_drop, axis=1, inplace=True) 
                                # удаляю именно по оси 1, то есть ненужные столбцы
X_test.drop(to_drop, axis=1, inplace=True)

In [ ]:
y = X_train['salary_class'] # ответы для обучающей выборки
X_train.drop('salary_class', axis=1,inplace=True) # удаляю таргет ответы из выборки, т.к. веса настраиваются не по ним

In [23]:
print(X_train.isnull().sum(), '\n\n')
print(X_test.isnull().sum())

hackathons_count                      0
courses_count                      1515
pet_projects_count                    0
leetcode_solved                       0
hours_study_per_week                  0
months_with_mentor                    0
gpa                                1011
internship                            0
is_mentor                             0
english_level                      3161
specialization                        0
communication_score                1961
teamworks_score                    1961
motivation_score                      0
consistency_score                     0
head_of_department                    0
hackathons_winner                     0
events_attended                       0
was_speaker                           0
bot_month                             0
coffee_cups_per_day                   0
github_commits_last_year              0
years_of_programming_experience       0
hours_sleep_per_day                   0
stress_score                          0


In [24]:
# Делю признаки на числовые и категориальные
num_feats = X_train.select_dtypes(include=[int, float]).columns.tolist()
cat_feats = X_train.select_dtypes(include=['object', 'category']).columns.tolist()


print(f"Числовые колонки: {num_feats}")
print(f"Категориальные колонки: {cat_feats}")

Числовые колонки: ['hackathons_count', 'courses_count', 'pet_projects_count', 'leetcode_solved', 'hours_study_per_week', 'months_with_mentor', 'gpa', 'communication_score', 'teamworks_score', 'motivation_score', 'consistency_score', 'events_attended', 'coffee_cups_per_day', 'github_commits_last_year', 'years_of_programming_experience', 'hours_sleep_per_day', 'stress_score', 'job_search_duration_months', 'ml_theory_score', 'coding_skill', 'math_score']
Категориальные колонки: ['internship', 'is_mentor', 'english_level', 'specialization', 'head_of_department', 'hackathons_winner', 'was_speaker', 'bot_month', 'attends_club_meetups_regularly', 'takes_part_in_memes', 'takes_part_in_informal_meetings']


In [25]:
# Заполняю пропуски средним по каждому столбцу среди числовых признаков с пропусками
for feat in num_feats:
    if X_train[feat].isnull().any():
        mean = X_train[feat].mean()
        X_train[feat] = X_train[feat].fillna(mean)
        print(f'в train признак {feat} заполнен значением {mean}')
        
    if X_test[feat].isnull().any():
        mean = X_test[feat].mean()
        X_test[feat] = X_test[feat].fillna(mean)
        print(f'в test признак {feat} заполнен значением {mean}')

в train признак courses_count заполнен значением 8.016914049016224
в test признак courses_count заполнен значением 7.953532608695652
в train признак gpa заполнен значением 3.527942481234107
в test признак gpa заполнен значением 3.508271855554688
в train признак communication_score заполнен значением 5.009136638874625
в test признак communication_score заполнен значением 4.951807290365945
в train признак teamworks_score заполнен значением 5.016704774286869
в test признак teamworks_score заполнен значением 4.983209446608549


In [26]:
# Заполненяю пропуски модой по каждому столбцу среди категориальных признаков с пропусками
for feat in cat_feats:
    if X_train[feat].isnull().any():
        mode_val = X_train[feat].mode()[0] if not X_train[feat].mode().empty else 'Unknown'
        X_train[feat] = X_train[feat].fillna(mode_val)
        print(f"в train признак '{feat}': заполнен модой {mode_val}")

    if X_test[feat].isnull().any():
        mode_val = X_test[feat].mode()[0] if not X_test[feat].mode().empty else 'Unknown'
        X_test[feat] = X_test[feat].fillna(mode_val)
        print(f"в train признак '{feat}': заполнен модой {mode_val}")

в train признак 'english_level': заполнен модой B1
в train признак 'english_level': заполнен модой B1


In [27]:
# теперь пропусков нигде нет

print(X_train.isnull().sum(), '\n\n')

print(X_test.isnull().sum())

hackathons_count                   0
courses_count                      0
pet_projects_count                 0
leetcode_solved                    0
hours_study_per_week               0
months_with_mentor                 0
gpa                                0
internship                         0
is_mentor                          0
english_level                      0
specialization                     0
communication_score                0
teamworks_score                    0
motivation_score                   0
consistency_score                  0
head_of_department                 0
hackathons_winner                  0
events_attended                    0
was_speaker                        0
bot_month                          0
coffee_cups_per_day                0
github_commits_last_year           0
years_of_programming_experience    0
hours_sleep_per_day                0
stress_score                       0
job_search_duration_months         0
attends_club_meetups_regularly     0
t